# Transfer Learning - VGGNet

**Załadujemy gotowy model (w tym przypadku VGGNet19) i dostroimy go do nowego zadania klasyfikacji binarnej - przykład z książki "Uczenie głębokie i sztuczna inteligencja. Interaktywny przewodnik ilustrowany"**

### Ładujemy zależności

In [66]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Ładujemy wytrenowany model VGG19

In [67]:
vgg19 = VGG19(include_top=False, #ostatnie warstwy zagęszczone, specyficzne dla zbioru, nie zostają załadowane
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

### "Zamrażamy" warstwy modelu VGGNet19 - nie będą się douczać

In [68]:
for layer in vgg19.layers:
    layer.trainable = False

### Tworzymy całościowy model, "u góry" dodajemy VGGNet19, a dalej własne warstwy

**Zadanie 3. Dodaj warstwę spłaszczającą, dropout 0.5 i warstwę pozwalającą sklasyfikować binarnie obiekty**

In [69]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [70]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_6 (Flatten)         (None, 25088)             0         
                                                                 
 dropout_6 (Dropout)         (None, 25088)             0         
                                                                 
 dense_6 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 20074562 (76.58 MB)
Trainable params: 50178 (196.01 KB)
Non-trainable params: 20024384 (76.39 MB)
_________________________________________________________________


### Kompilujemy model

In [71]:
model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

### Pobierz zbiór danych z [Kaggle'a](https://www.kaggle.com/datasets/dansbecker/hot-dog-not-hot-dog) i rozpakuj

### Zbiór jest mały, więc tworzymy instancje klasy ImageDataGenerator, które pozwolą nam powiększyć dane

In [72]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

### Deklarujemy rozmiar paczki

In [73]:
batch_size = 32

### Powiększamy dane

In [74]:
train_generator = train_datagen.flow_from_directory(
    directory= './train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory= './test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


**Zadanie 4. Wypisz co oznaczają wszystkie zadeklarowane parametry generatorów danych**

tu odpowiedź

### Uczymy model z danymi z generatora (dla ułatwienia używamy do tego oddzielnej funkcji, choć będzie w następnej wersji tensorflowa usunięta)

In [75]:
model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)

/tmp/ipykernel_7690/2768083069.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, steps_per_epoch=15, epochs=16, validation_data=valid_generator, validation_steps=15)


Epoch 1/16
15/15 [==============================] - ETA: 0s - loss: 1.1119 - accuracy: 0.5601 

KeyboardInterrupt: 